In [7]:
# Instalar Detectron2 desde el repositorio oficial
!pip install torch torchvision

# Asegurarte de que tienes PyCOCO tools para trabajar con datasets etiquetados en formato COCO
!pip install pycocotools


In [8]:
!git clone https://github.com/facebookresearch/detectron2.git


fatal: destination path 'detectron2' already exists and is not an empty directory.


In [1]:
!pip install -e /content/detectron2


Obtaining file:///content/detectron2
  Preparing metadata (setup.py) ... done
  Attempting uninstall: detectron2
    Found existing installation: detectron2 0.6
    Uninstalling detectron2-0.6:
      Successfully uninstalled detectron2-0.6
  Running setup.py develop for detectron2


In [2]:
from detectron2.config import get_cfg

cfg = get_cfg()
cfg.merge_from_file("/content/detectron2/configs/COCO-Detection/fast_rcnn_R_50_FPN_1x.yaml")



In [3]:
# Descargar y descomprimir el dataset desde Roboflow
!cd /content/yolov5/detectron2
!curl -L "https://app.roboflow.com/ds/fcgsu1iErW?key=G1glPBvFxa" > roboflow.zip
!unzip roboflow.zip -d /content/detectron2/datasets


/bin/bash: line 1: cd: /content/yolov5/detectron2: No such file or directory
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   901  100   901    0     0   2960      0 --:--:-- --:--:-- --:--:--  2963
100  148M  100  148M    0     0  59.5M      0  0:00:02  0:00:02 --:--:-- 74.6M
Archive:  roboflow.zip
checkdir:  cannot create extraction directory: /content/yolov5/detectron2/datasets
           No such file or directory


In [4]:
!unzip roboflow.zip -d /content/detectron2/datasets

Archive:  roboflow.zip
  inflating: /content/detectron2/datasets/README.dataset.txt  
  inflating: /content/detectron2/datasets/README.roboflow.txt  
   creating: /content/detectron2/datasets/test/
  inflating: /content/detectron2/datasets/test/_annotations.coco.json  
 extracting: /content/detectron2/datasets/test/trucha_004_jpg.rf.b83b58fe86f90df6c30f6332c072ce80.jpg  
 extracting: /content/detectron2/datasets/test/trucha_014_jpg.rf.0fdc09078fc782d39dcf232922c50ae2.jpg  
 extracting: /content/detectron2/datasets/test/trucha_029_jpg.rf.48c26d46f2f682cf34b956d1f08c8785.jpg  
 extracting: /content/detectron2/datasets/test/trucha_039_jpg.rf.37d8d730b4ac554d11ab0c5420a18dc9.jpg  
 extracting: /content/detectron2/datasets/test/trucha_050_jpg.rf.e4ef8b763d4eb200754ef1362b176d36.jpg  
 extracting: /content/detectron2/datasets/test/trucha_066_jpg.rf.bf283a0f57af581dd9aff3e900f2f6a1.jpg  
 extracting: /content/detectron2/datasets/test/trucha_080_jpg.rf.f1a1b412a8c72f7e7d7503e0129127a1.jpg  
 e

In [5]:
import detectron2
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.evaluation import COCOEvaluator
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
import os
import numpy as np
import pandas as pd  # Importamos pandas para manejar el CSV

# Crear lista para almacenar todas las métricas de cada época
epoch_metrics = []

# Verificar si los datasets ya están registrados antes de intentar eliminarlos
if "train_dataset" in DatasetCatalog:
    DatasetCatalog.remove("train_dataset")
if "val_dataset" in DatasetCatalog:
    DatasetCatalog.remove("val_dataset")

# Registrar el dataset en formato COCO
register_coco_instances("train_dataset", {}, "/content/detectron2/datasets/train/_annotations.coco.json", "/content/detectron2/datasets/train")
register_coco_instances("val_dataset", {}, "/content/detectron2/datasets/valid/_annotations.coco.json", "/content/detectron2/datasets/valid")

# Configuración de los metadatos
train_metadata = MetadataCatalog.get("train_dataset")
val_metadata = MetadataCatalog.get("val_dataset")

# Configuración del modelo
cfg = get_cfg()
cfg.merge_from_file("/content/detectron2/configs/COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")  # Cargar archivo de configuración

# Ruta del dataset y pesos preentrenados
cfg.DATASETS.TRAIN = ("train_dataset",)
cfg.DATASETS.TEST = ("val_dataset",)  # Para evaluación
cfg.DATALOADER.NUM_WORKERS = 4

# Hiperparámetros
cfg.SOLVER.IMS_PER_BATCH = 8  # Batch size a 8
cfg.SOLVER.BASE_LR = 0.0025    # Learning rate inicial
cfg.SOLVER.MAX_ITER = 50 * (len(DatasetCatalog.get("train_dataset")) // 8)  # 50 epochs, ajustado al tamaño del dataset
cfg.SOLVER.STEPS = []          # Sin reducción de LR durante el entrenamiento

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 8  # Batch size por imagen
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 6  # Número de clases en tu dataset

# Cargar pesos preentrenados desde la URL
cfg.MODEL.WEIGHTS = "detectron2://COCO-Detection/faster_rcnn_R_50_FPN_3x/137849458/model_final_280758.pkl"

# Tamaño de la imagen
cfg.INPUT.MIN_SIZE_TRAIN = 640
cfg.INPUT.MIN_SIZE_TEST = 640

# Guardar el modelo entrenado
cfg.OUTPUT_DIR = "/content/detectron2/output"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# Entrenamiento con evaluación
class Trainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        return COCOEvaluator(dataset_name, cfg, False, output_folder)

    # Sobrescribir la función de entrenamiento para almacenar todas las métricas por cada época
    def run_step(self):
        super().run_step()

        # Obtener las métricas después de cada epoch
        metrics = self.storage.history  # Aquí debería estar el diccionario de métricas

        # Asegúrate de que `metrics` es un diccionario y no una función
        if isinstance(metrics, dict):
            epoch_metric = {
                'epoch': self.storage.iter,
                'train/box_loss': metrics.get("train/box_loss", {}).get('avg', np.nan),
                'train/obj_loss': metrics.get("train/obj_loss", {}).get('avg', np.nan),
                'train/cls_loss': metrics.get("train/cls_loss", {}).get('avg', np.nan),
                'metrics/precision': metrics.get("metrics/precision", {}).get('avg', np.nan),
                'metrics/recall': metrics.get("metrics/recall", {}).get('avg', np.nan),
                'metrics/mAP_0.5': metrics.get("metrics/mAP_0.5", {}).get('avg', np.nan),
                'metrics/mAP_0.5:0.95': metrics.get("metrics/mAP_0.5:0.95", {}).get('avg', np.nan),
                'val/box_loss': metrics.get("val/box_loss", {}).get('avg', np.nan),
                'val/obj_loss': metrics.get("val/obj_loss", {}).get('avg', np.nan),
                'val/cls_loss': metrics.get("val/cls_loss", {}).get('avg', np.nan),
                'x/lr0': metrics.get("x/lr0", {}).get('avg', np.nan),
                'x/lr1': metrics.get("x/lr1", {}).get('avg', np.nan),
                'x/lr2': metrics.get("x/lr2", {}).get('avg', np.nan),
            }
            epoch_metrics.append(epoch_metric)

# Entrenar el modelo
trainer = Trainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

# Evaluar el modelo después del entrenamiento
evaluator = COCOEvaluator("val_dataset", cfg, False, output_dir=os.path.join(cfg.OUTPUT_DIR, "inference"))
trainer.test(cfg, trainer.model, evaluators=[evaluator])

# Guardar todas las métricas por cada época en un archivo CSV
metrics_df = pd.DataFrame(epoch_metrics)
metrics_df.to_csv('/content/detectron2/output/all_metrics_per_epoch.csv', index=False)
print("Todas las métricas por época guardadas en 'all_metrics_per_epoch.csv'.")


Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.



[12/02 03:30:23 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
model_final_280758.pkl: 167MB [00:00, 168MB/s]                           
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}


[12/02 03:30:25 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.10/dist-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[12/02 03:30:46 d2.utils.events]:  eta: 2:58:17  iter: 19  total_loss: 3.069  loss_cls: 2  loss_box_reg: 0.9843  loss_rpn_cls: 0.06278  loss_rpn_loc: 0.02134    time: 0.8204  last_time: 0.8155  data_time: 0.0899  last_data_time: 0.0597   lr: 4.9952e-05  max_mem: 4703M
[12/02 03:31:06 d2.utils.events]:  eta: 2:59:37  iter: 39  total_loss: 2.089  loss_cls: 1.095  loss_box_reg: 0.9013  loss_rpn_cls: 0.03587  loss_rpn_loc: 0.01913    time: 0.8374  last_time: 0.8203  data_time: 0.0781  last_data_time: 0.0652   lr: 9.9902e-05  max_mem: 4703M
[12/02 03:31:23 d2.utils.events]:  eta: 3:02:44  iter: 59  total_loss: 1.714  loss_cls: 0.7794  loss_box_reg: 0.909  loss_rpn_cls: 0.01262  loss_rpn_loc: 0.01666    time: 0.8423  last_time: 0.8264  data_time: 0.0800  last_data_time: 0.0547   lr: 0.00014985  max_mem: 4703M
[12/02 03:31:40 d2.utils.events]:  eta: 3:03:50  iter: 79  total_loss: 1.575  loss_cls: 0.6775  loss_box_reg: 0.9088  loss_rpn_cls: 0.007493  loss_rpn_loc: 0.01524    time: 0.8480  last

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


[12/02 06:50:27 d2.evaluation.evaluator]: Inference done 11/87. Dataloading: 0.0062 s/iter. Inference: 0.0619 s/iter. Eval: 0.0004 s/iter. Total: 0.0685 s/iter. ETA=0:00:05
[12/02 06:50:31 d2.evaluation.evaluator]: Total inference time: 0:00:05.160314 (0.062931 s / iter per device, on 1 devices)
[12/02 06:50:31 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:04 (0.059414 s / iter per device, on 1 devices)
[12/02 06:50:31 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[12/02 06:50:31 d2.evaluation.coco_evaluation]: Saving results to /content/detectron2/output/inference/coco_instances_results.json
[12/02 06:50:31 d2.evaluation.coco_evaluation]: Evaluating predictions with unofficial COCO API...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
[12/02 06:50:31 d2.evaluation.fast_eval_api]: Evaluate annotation type *bbox*
[12/02 06:50:31 d2.evaluation.fast_eval_api]: COCOeval_opt.evaluate() finished in 0.02 seconds.
[1

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


[12/02 06:50:32 d2.evaluation.evaluator]: Inference done 11/87. Dataloading: 0.0009 s/iter. Inference: 0.0591 s/iter. Eval: 0.0003 s/iter. Total: 0.0603 s/iter. ETA=0:00:04
[12/02 06:50:37 d2.evaluation.evaluator]: Total inference time: 0:00:05.088256 (0.062052 s / iter per device, on 1 devices)
[12/02 06:50:37 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:04 (0.059077 s / iter per device, on 1 devices)
[12/02 06:50:37 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[12/02 06:50:37 d2.evaluation.coco_evaluation]: Saving results to /content/detectron2/output/inference/coco_instances_results.json
[12/02 06:50:37 d2.evaluation.coco_evaluation]: Evaluating predictions with unofficial COCO API...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
[12/02 06:50:37 d2.evaluation.fast_eval_api]: Evaluate annotation type *bbox*
[12/02 06:50:37 d2.evaluation.fast_eval_api]: COCOeval_opt.evaluate() finished in 0.03 seconds.
[1

In [6]:
import shutil
from google.colab import files

# Comprimir la carpeta
shutil.make_archive('/content/detectron2_output', 'zip', '/content/detectron2/output')

# Descargar el archivo comprimido
files.download('/content/detectron2_output.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

v14